## Cross Validation on our dataset using Baseline and RNN models

### importing required libraries

In [12]:
# Data manipulation
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

# Data Retrieval
from power.ml_ops.data import clean_pv_data, get_pv_data

# System
import os

# Manipulating temporal data and check the types of variables
from typing import Dict, List, Tuple, Sequence

# tensforflow
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.layers import Lambda

First we import our clean dataset

In [13]:
pv_raw_data = get_pv_data()
pv_df = clean_pv_data(pv_raw_data)
pv_df.rename(columns={'electricity': 'power'}, inplace=True)
pv_df = pv_df[['power']]
pv_df.head(3)

# data loaded
# data cleaned


,power
0,0.0
1,0.0
2,0.0


### Creating FOLDs

In [2]:
def get_folds(
    df: pd.DataFrame,
    fold_length: int,
    fold_stride: int) -> List[pd.DataFrame]:
    '''
    This function slides through the Time Series dataframe of shape (n_timesteps, n_features) to create folds
    - of equal `fold_length`
    - using `fold_stride` between each fold

    Returns a list of folds, each as a DataFrame
    '''
    folds = []
    for idx in range(0, len(df), fold_stride):
        if (idx + fold_length) > len(df):
            break
        fold = df.iloc[idx:idx + fold_length, :]  # select from row idx til last row of the fold (6 years), all the columns
        folds.append(fold)                        # append the 6 year fold to folds
    return folds

### Splitting folds

In [3]:
def train_test_split(fold:pd.DataFrame,
                     train_test_ratio: float,
                     input_length: int) -> Tuple[pd.DataFrame]:
    '''
    Returns a train dataframe and a test dataframe (fold_train, fold_test)
    from which one can sample (X,y) sequences.
    df_train should contain all the timesteps until round(train_test_ratio * len(fold))
    '''
    # TRAIN SET
    # ======================
    last_train_idx = round(train_test_ratio * len(fold))    # 83% of the fold for train
    fold_train = fold.iloc[0:last_train_idx, :]             # 1st until last row of train set, all columns

    # TEST SET
    # ======================
    first_test_idx = last_train_idx - input_length          # last row of train set - 2 weeks --> test set starts 2 weeks
                                                            # before train set ends --> overlap (not a problem with X)
    fold_test = fold.iloc[first_test_idx:, :]               # 1st until last row of test set, all columns

    return (fold_train, fold_test)

### Creating sequences randomly

First we create a random sequence and then we generate the whole number of sequences of our choice. This function will return two sets of 3D numpy arrays for our inputs and outputs of the model. 

In [16]:
def get_Xi_yi(
    fold:pd.DataFrame,
    input_length:int,       # 48
    output_length:int):     # 24
    '''
    - given a fold, it returns one sequence (X_i, y_i)
    - with the starting point of the sequence being chosen at random
    '''
    first_possible_start = 0                                                    # the +1 accounts for the index, that is exclusive.
    last_possible_start = len(fold) - (input_length + output_length) + 1        # It can start as long as there are still
                                                                                # 48 + 1 days after the 1st day.
    random_start = np.random.randint(first_possible_start, last_possible_start) # np.random to pick a day inside the possible interval.

    X_i = fold.iloc[random_start:random_start+input_length]
    y_i = fold.iloc[random_start+input_length:
                  random_start+input_length+output_length][[TARGET]]            # creates a pd.DataFrame for the target y

    return (X_i, y_i)

def get_X_y(
    fold:pd.DataFrame,
    number_of_sequences:int,
    input_length:int,
    output_length:int):

    X, y = [], []                                                 # lists for the sequences for X and y

    for i in range(number_of_sequences):
        (Xi, yi) = get_Xi_yi(fold, input_length, output_length)   # calls the previous function to generate sequences X + y
        X.append(Xi)
        y.append(yi)

    return np.array(X), np.array(y)

### Model Initialization

#### * Baseline

In [8]:
def init_baseline():

    model = models.Sequential()
    # a layer to take the last value of the sequence and output it
    model.add(layers.Lambda(lambda x: x[:,-25:-1,0,None]))                      # all sequences, last day, 1 feature (pv_power)


    adam = optimizers.Adam(learning_rate=0.02)
    model.compile(loss='mse', optimizer=adam, metrics=["mae"])

    return model

#### * RNN model 

In [23]:
def init_model(X_train, y_train, n_unit=24, learning_rate=0.02):


    # 1 - RNN architecture
    # ======================
    model = models.Sequential()

    ## 1.1 - Recurrent Layer
    model.add(layers.LSTM(n_unit,
                          activation='tanh',
                          return_sequences = False,
                          input_shape=(X_train.shape[1],X_train.shape[2])
                          ))
    ## 1.2 - Predictive Dense Layers
    output_length = y_train.shape[1]
    model.add(layers.Dense(output_length, activation='linear'))

    # 2 - Compiler
    # ======================
    adam = optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=adam, metrics=["mae"])

    return model

### Cross Validation

In [24]:
# Specify the parameters

TARGET = 'power'
fold_length = 52560             # 6 years
fold_stride = 52560             # 6 years
train_test_ratio = 0.83         # 5 yrs/6 yrs
input_length = 48               # number of obsevations per one sequence
output_length = 24              # Day-ahead predictions
n_seq_train = 250               # number_of_sequences_train
n_seq_test = 50                 # number_of_sequences_test
n_unit = 24                     # number of hidden units
learning_rate = 0.02
patience = 5
epochs = 50
batch_size = 32

In [28]:
def cross_validate_baseline_and_lstm():

    list_of_mae_baseline_model = []
    list_of_mae_recurrent_model = []

    # 0 - Creating folds
    # =========================================
    folds = get_folds(pv_df, fold_length, fold_stride)  # function we coded to get the folds

    for fold_id, fold in enumerate(folds):

        # 1 - Train/Test split the current fold
        # =========================================
        (fold_train, fold_test) = train_test_split(fold, train_test_ratio, input_length) # function we coded to split train/test

        X_train, y_train = get_X_y(fold_train, n_seq_train, input_length, output_length)
        X_test, y_test = get_X_y(fold_test, n_seq_test, input_length, output_length)

        # 2 - Modelling
        # =========================================

        ##### Baseline Model
        baseline_model = init_baseline()
        mae_baseline = baseline_model.evaluate(X_test, y_test, verbose=0)[1]   # evaluating baseline model (metric)
        list_of_mae_baseline_model.append(mae_baseline)
        print("-"*50)
        print(f"MAE baseline fold n°{fold_id} = {round(mae_baseline, 2)}")

        ##### LSTM Model
        model = init_model(X_train, y_train, n_unit, learning_rate)
        es = EarlyStopping(monitor = "val_mae",
                           mode = "min",
                           patience = patience,
                           restore_best_weights = True)
        history = model.fit(X_train, y_train,
                            validation_split = 0.3,
                            shuffle = False,
                            batch_size = batch_size,
                            epochs = epochs,
                            callbacks = [es],
                            verbose = 0)
        res = model.evaluate(X_test, y_test, verbose=0)    # evaluating LSTM (metric)
        mae_lstm = res[1]
        list_of_mae_recurrent_model.append(mae_lstm)
        print(f"MAE LSTM fold n°{fold_id} = {round(mae_lstm, 2)}")

        ##### Comparison LSTM vs Baseline for the current fold
        print(f"improvement over baseline: {round((1 - (mae_lstm/mae_baseline))*100,2)} % \n")

    return list_of_mae_baseline_model, list_of_mae_recurrent_model


In [29]:
mae_baselines, mae_lstms = cross_validate_baseline_and_lstm()

--------------------------------------------------
MAE baseline fold n°0 = 0.07
MAE LSTM fold n°0 = 0.07
improvement over baseline: 8.0 % 

--------------------------------------------------
MAE baseline fold n°1 = 0.07
MAE LSTM fold n°1 = 0.07
improvement over baseline: 6.71 % 

--------------------------------------------------
MAE baseline fold n°2 = 0.07
MAE LSTM fold n°2 = 0.07
improvement over baseline: -3.89 % 

--------------------------------------------------
MAE baseline fold n°3 = 0.06
MAE LSTM fold n°3 = 0.06
improvement over baseline: -0.23 % 

--------------------------------------------------
MAE baseline fold n°4 = 0.07
MAE LSTM fold n°4 = 0.06
improvement over baseline: 16.95 % 

--------------------------------------------------
MAE baseline fold n°5 = 0.07
MAE LSTM fold n°5 = 0.06
improvement over baseline: 3.34 % 

--------------------------------------------------
MAE baseline fold n°6 = 0.07
MAE LSTM fold n°6 = 0.07
improvement over baseline: 5.96 % 



In [30]:
print(f"average percentage improvement over baseline = {round(np.mean(1 - (np.array(mae_lstms)/np.array(mae_baselines))),2)*100}%")

average percentage improvement over baseline = 5.0%
